In [ ]:
import sys
import os
import json
import time
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from nlp_utils.src.nlputils import utils
from nlp_utils.src.nlputils.components.docling_util import doclingserver

c:\Users\Prashant\anaconda3\envs\nlpdoc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
###### Declare the variables #######

path_to_files_list = "path to read files"


In [3]:
# read the files dataframe
df_pdf = pd.read_json(path_to_files_list)\
                    .query('ImagePDF.notna() & ImagePDF==False').reset_index(drop=True)
df_pdf['pdf_path'] = df_pdf.apply(lambda x: x['orig_filepath'] if x['filetype'] == 'pdf' 
                                  else x['converted_filepath'],axis=1)
df_pdf = df_pdf[df_pdf.pdf_path.notna()]\
                    .sort_values(by = 'page_count',ascending=True)\
                    .reset_index(drop=True)
print("normal pdf files to be processed:",len(df_pdf))

# keeping image pdf as separate dataframe
df_imagepdf = pd.read_json(path_to_files_list)\
                    .query('ImagePDF.notna() & ImagePDF==True').reset_index(drop=True)
df_imagepdf['pdf_path'] = df_imagepdf.apply(lambda x: 
                                x['orig_filepath'] if x['filetype'] == 'pdf' 
                                else x['converted_filepath'],axis=1)
df_imagepdf = df_imagepdf[df_imagepdf.pdf_path.notna()]\
                    .sort_values(by = 'page_count',ascending=True)\
                    .reset_index(drop=True)
print("image pdf files to be processed:",len(df_imagepdf))

normal pdf files to be processed: 154
image pdf files to be processed: 42


In [ ]:
success_count, partial_success_count, failure_count, folder_info= doclingserver.batch_processing(
                                    file_list=df_pdf.pdf_path.tolist(),
                                    output_dir='path to save output')

Process image pdfs

In [ ]:
for file in tqdm(df_imagepdf.pdf_path.to_list()):
    a,b = doclingserver.useOCR(file)
    c,d = doclingserver.save_output(a, 'path to save output' , b)